In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import show_doc

# Directives

> A cheat sheet of directives available in nbdev.

Directives are special comments that are preceeded by `#|` that control:

1. Cell visibility in rendered documentation
2. Execution of cells for tests and docs
3. How source code is generated from notebook cells

Quarto contains many directives for categories 1 and 2.  nbdev augments Quarto by adding more directives for all three categories.  All Quarto directives can be used in nbdev notebooks.  

This cheat sheet lists all nbdev directives in addition to some Quarto directives we believe are important.  We recommend consulting the [quarto docs](https://quarto.org/docs/reference/cells/cells-jupyter.html) to see all of the directives available to you.

To clarify the origin of directives we use the following emojis:
- 📔 for nbdev-only directives 
- ❤️ for Quarto directives (which also work in nbdev)

## Cell Visibility

The following directives control cell visibility in rendered documentation:

### 📔 `#|hide`

When you use this directive, you will not see the cell input or output. 

<details>
  <summary>See Example</summary>

For example, the following input will result in the contents of the cell and it's output from being hidden:

```python
#|hide
print('you will not see this')
```

Note that using `#|hide` is equivalent to using the following two directives together in a cell:

```python
#| echo: false
#| output: false
print('you will not see this')
```

The `#| echo` and `#| output` directives are discussed in a later section.
    
</details>

### ❤️ `#|echo: <true|false>`

Toggle the visibility of code-cell inputs.

<details>
  <summary>See Example</summary>

```python
#|echo: false
print('you can see the output but not the code!')
```

    
which results in:
    
    
```
you can see the output but not the code!
```
    
</details>

### ❤️ `#|output: <true|false|asis>`

Setting this to `false` hides the output of a cell. Setting this to `asis` renders the output as raw markdown. See the [Quarto docs](https://quarto.org/docs/reference/cells/cells-jupyter.html#cell-output) for more information.

<details>
  <summary>See Example of <code>#|output: false</code></summary>
  
The following cell will not display any output:

```python
#|output: false
1 + 1
```
</details>
<details>
  <summary>See Example of <code>#|output: asis</code></summary>
    
The following cell with `#|output: asis` will produce the output `hello fastai` rendered as markdown instead of a string:  
    
```python    
#|output: asis
print("`hello fastai`")
```         
</details>

### 📔 `#|hide_line`

You can use this to hide as specific line in your code.  

<details>
  <summary>See Example</summary>

For example:

```python
def _secret(): ...

for i in range(3):
    _secret() #|hide_line
    print(i)
```

becomes this:

In [ ]:
def _secret(): ...

for i in range(3):
    _secret() #|hide_line
    print(i)

0
1
2


</details>

### 📔 `#|filter_stream <keyword> ...`

This allows you to filter lines containing specific keywords in cell outputs.  For example


```python
#|filter_stream FutureWarning MultiIndex
print('\n'.join(['A line', 'Foobar baz FutureWarning blah', 
                 'zig zagMultiIndex zoom', 'Another line.']))
```

will output this:

In [ ]:
#|echo: false
#|filter_stream FutureWarning MultiIndex
print('\n'.join(['A line', 'Foobar baz FutureWarning blah', 
                 'zig zagMultiIndex zoom', 'Another line.']))

A line
Foobar baz FutureWarning blah
zig zagMultiIndex zoom
Another line.


### ❤️ `#|code-fold: <show|true>`

The `#|code-fold` directive allows you to collapse code (note: must use  `code-fold` and not `code_fold`):

```python
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")
```

which results in (click to expand):

In [ ]:
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")

With `true` it is collapsed by default, with `show` it is shown by default and still collapsible. The directive also works with `#|exports` but not with `#|export`:

```python
#|exports
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")
```

which results in (click to expand):

In [ ]:
#|echo: true
#|code-fold: true
def foo():
    print("this")
    print("is")
    print("a")
    print("long")
    print("body")

## Generating Source Code

### 📔 `#|default_exp <name>`

Required for every notebook. This directive names the module where things will be exported. For example:

```python
#| default_exp baz

# In a new notebook cell:

#| export
def my_function(): pass
```

If our package is named: `bitsnbytes` then we can do:

```python
from bitsnbytes.baz import my_function
```

You can define the package name by using `lib_name` in `settings.ini`.

### 📔 `#|export`

Exports the items in the cell into the generated module and documentation. For example:

```python
#|export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'
```

The documentation will look like this:

In [ ]:
#|exec_doc
#|echo: false
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#|echo: false
# Note: we are using show_doc like this to simulate the effect of #|export without using export
show_doc(say_hello)

---

### say_hello

>      say_hello (to)

Say hello to somebody

The docs are generated from this export using `showdoc.show_doc`. For a detailed discussion of `showdoc.show_doc` see [here](https://nbdev.fast.ai/explanations/showdoc.html). The exports are automatically included in `__all__` for the module.

### 📔 `#|export <some.thing>`

Similar to `#|export` above. Instead of exporting to the module named by `#|default_exp` export to the specified module.

If our package is named: `bitsnbytes`, and we have previously included: `#|default_exp core` in this notebook, and we have an existing notebook with `#|default_exp bar`, then:

```python
# Earlier in the notebook:

#|default_exp core

# A new notebook cell:

#|export bar
def foo(): pass
```

then we can import this as:
```python
from bitsnbytes.bar import foo
```

### 📔 `#|exporti`

An `i`nternal export. Not included in `__all__` or the docs. Useful for a function that is called by other functions in this module but is not part of the public API.

Equivalently you can prefix your function or method with `_` e.g. `def _private(): pass`.

### 📔 `#|exports`

A `s`ource export. Like `#|export` but in addition to showing docs via `showdoc.show_doc`, it also shows the source code. Given the following code:

```python
#|exports
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'
```

this will produce the following output:

In [ ]:
#|exec_doc
#|echo: true
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#|echo: false
show_doc(say_hello)

---

### say_hello

>      say_hello (to)

Say hello to somebody

## Cell Execution

### 📔 `#|exec_doc`

In [ ]:
#|hide
import datetime

Ensures that a cell is executed each time before generating docs. When a cell does not have this annotation, it is run according to the default rules [described here](https://nbdev.fast.ai/explanations/showdoc.html). 

In [ ]:
datetime.datetime.now()

datetime.datetime(2022, 8, 17, 23, 21, 42, 175324)

However with the annotation:
```python
#|exec_doc
datetime.datetime.now()
```

we can see that the time has been updated:

In [ ]:
#|exec_doc
datetime.datetime.now()

datetime.datetime(2022, 8, 17, 23, 21, 42, 572294)

### 📔 `#|notest`

In [ ]:
#|notest
assert 1 == 2

Do not run this cell during testing. This is the default test flag that [comes with nbdev](https://github.com/fastai/nbdev/blob/cdd1187d9d4d130197259143ad00bdeb9c4266d4/settings.ini#L51).

It is possible to specify additional values. For example see [fastai](https://github.com/fastai/fastai/blob/c5b9aa050e1ed382b40a7f772a07d74453fdcacb/settings.ini#L27). This allows you to toggle certain tests on and off by running `nbdev_test --flags`.

### ❤️ `#|eval: <true|false>`

This quarto directive is respected by `nbdev`.  When set to `false`, this cell is ignored during testing.  The `#|notest` directive gives you more fine-grained control over testing.

Example:

```python
#|eval: false
raise Exception("I'm not raised because I'm not run")
```

### 📔 `no directive`


For completeness sake, when a cell has no directive applied to it e.g. the cell does not include any line that starts with `#|`, then the cell:

* is show in the docs as-is,
* manually added `showdoc.show_doc` cells are run automatically,
* executes in tests,
* if it contains `import x` or `from x import y` then the cell is run

See [this page](https://nbdev.fast.ai/explanations/showdoc.html) for more.